# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>


# <font color="#003660">Week 1: Introduction to Natural Language Processing</font>

# <font color="#003660">Notebook 3: Regression</font>

<center><br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/dag.png"/><br></center>

<p>
<center>
<div>
    <font color="#085986"><b>By the end of this lesson, you will be able to...</b><br><br>
        ... transform raw text into a term-document matrix, <br>
        ... train a regression model on the term-document matrix, and <br> ... and compete in a Kaggle competition.
    </font>
</div>
</center>
</p>

# Import packages

As always, we first need to load a number of required Python packages:
- `pandas` provides high-performance, easy-to-use data structures and data analysis tools.
- `SQLAlchemy`, together with `pymysql`, allows to communicate with SQL databases.
- `getpass` provides function to safely enter passwords.
- `spacy` offers industrial-strength natural language processing.
- `sklearn` is the de-facto standard machine learning package in Python.

In [ ]:
# Install packages
!pip install pymysql

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import getpass
import spacy
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load documents

We load our data from a MySQL database. For security reasons, we don't store the database credentials here; please have a look at Panda to get them.

In [ ]:
# Get credentials
user = input("Username: ")
passwd = getpass.getpass("Password: ")
server = input("Server: ")
db = input("Database: ")

# Create an engine instance (SQLAlchemy)
engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format(user, passwd ,server, db))

# Define SQL query
sql_query = "SELECT * FROM WineDataset"

# Query dataset (pandas)
corpus = pd.read_sql(sql=sql_query, con=engine)

In [ ]:
corpus.head()

In [ ]:
corpus.shape

# Preprocess documents

Split data into training, validation, and test set.

In [ ]:
training = corpus[corpus["testset"] == 0]
validation = training.iloc[80000:100000,]
training = training.iloc[0:80000,]
test = corpus[corpus["testset"] == 1]

In [ ]:
print(training.shape)
print(validation.shape)
print(test.shape)

Perform standard NLP preprocessing steps on the training set using spaCy. To speed up things, we disable some components of spaCy's standard NLP pipeline.

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])

def spacy_prep_df(corpus):
  corpus = corpus.to_dict("records")
  for i, entry in enumerate(corpus):
      doc = nlp(entry[u'description'])
      tokens_to_keep = []
      for token in doc:
          if token.is_alpha and not token.is_stop:
              tokens_to_keep.append(token.lemma_.lower())
      entry[u'description_prep'] = " ".join(tokens_to_keep)
  corpus = pd.DataFrame(corpus)
  return(corpus)

In [ ]:
training = spacy_prep_df(training)

Display the first couple of lines of the preprocessed descriptions.

In [ ]:
training["description_prep"].head()

# Vectorize documents

Vectorize using a simple `CountVectorizer`.

In [ ]:
count_vect = CountVectorizer(min_df=10)

Apply the CountVectorizer object to the review texts of the training set.

In [ ]:
X_training = count_vect.fit_transform(training["description_prep"].tolist())

Store the labels that we want to predict in a separate variable.

In [ ]:
y_training = training["points"]
y_training.describe()

# Train regressor on training set

Fit a linear regression model with the term-document matrix as the features and the numeric wine quality (i.e., `points` variable) as the label.

In [ ]:
reg = LinearRegression().fit(X_training, y_training)

Test whether classifier is working by predicting the quality of a short fake review.

In [ ]:
doc_new = {'index': [1],
           'description': ['This is a good wine']}

doc_new_df = pd.DataFrame.from_dict(doc_new)

In [ ]:
doc_new_df_prep = spacy_prep_df(doc_new_df)
doc_new_df_prep

In [ ]:
X_new = count_vect.transform(doc_new_df_prep["description_prep"])
predicted = reg.predict(X_new)
predicted

# Evaluate accuracy on validation set

Before calculating the predictions of our model, let's first create a simple benchmark (i.e., always predicting the mean points of the training set).

In [ ]:
print(metrics.mean_absolute_error(y_validation, [88.43]*len(y_validation)))

Now let's evaluate the predictive accurcay of our model on the validation set.

In [ ]:
validation = spacy_prep_df(validation)

In [ ]:
X_validation = count_vect.transform(validation["description_prep"])
y_validation = validation["points"]

Call the predict function of our model with the validation data and calculate MAE.

In [ ]:
predictions_validation = reg.predict(X_validation)
print(metrics.mean_absolute_error(y_validation, predictions_validation))